# Logistic regression

We are going to do [logistic regression](https://en.wikipedia.org/wiki/Logistic_regression) on the famous [Iris dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set) using a neural network-like implementation, and plot the decision boundary.

# Setup
The topics of this exercise are very famous and very well known, so a lot of educational material is available online. E.g., for logistic regression:

* [Logistic regression @towardsdatascience](https://towardsdatascience.com/understanding-logistic-regression-9b02c2aec102)
* [Logistic regression @Biochemia Medica](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3936971/)
* [Logistic regression @datacamp](https://www.datacamp.com/community/tutorials/understanding-logistic-regression-python)

Let's just frame the terms of the first part of the problem:
* we are talking about **binary** classification: all samples belong to two classes (represented by values 0 and 1). We want to build a statistical machine able to predict, for a new sample, the class
* for each sample we have a set of numbers, called **features**. The easiest-to-understand case is when these features are actual phyisical measurements, but any set of data will do
* in **standard regression** we weight each feature with a number (positive or negative, big or small). We then multiply each feature times its weight and add everything...
* ...but we cannot do that now, since the resulting number could be very big (positive big or negative big). But our desired result is a class! In other words our machine needs to produce either a zero or a one. Or maybe **all the numbers between zero and one**, so that we have an idea about how certain our prediction is
* in fact, a 0.99 will show more confidence than a 0.8, which will give more confidence than a 0.65, even if all three outputs at the end can be considered as "class 1" (since those numbers are closer to 1 than to 0)
* to do so we feed what is produced by a regular regression into a **sigmoid function**, which looks like this:

<img src="https:///raw.githubusercontent.com/ne1s0n/coding_excercises/master/data/Logistic-curve.png" width="400">

This function takes in input any number from -infinity to +infinity and returns a value between zero and one. This will be our prediction. Let's start building the solution.

## Loading libraries and setting the random seed

First of all, we load some necessary libraries; then we setup the random seed to ensure reproducibility of results. Since tensorflow uses an internal random generator we need to fix both the general seed (via numpy `seed()`) and tensorflow seed (via `set_seet()`)

In [ ]:
#libraries
import numpy as np

In [ ]:
## setting seeds
from numpy.random import seed
seed(131)

import tensorflow as tf
#tf.random.set_seed(777)
tf.keras.utils.set_random_seed(10)
tf.config.experimental.enable_op_determinism()

# The Iris dataset

The dataset we are going to use is very famous. It was published by Robert Fisher in 1936 together with the paper [The use of multiple measurements in taxonomic problems](https://onlinelibrary.wiley.com/doi/abs/10.1111/j.1469-1809.1936.tb02137.x). Data are public and nowadays this dataset is shipped with many statistical software and packages. We are going to use the copy coming with [sci-kit learn](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html#sklearn.datasets.load_iris).

First of all, let's verify the sci-kit version available:

In [ ]:
import sklearn.datasets
print(sklearn.__version__)

From version 0.23+ we can simply do:

> `iris = sklearn.datasets.load_iris(return_X_y = True, as_frame = True)`

In [ ]:
#iris = sklearn.datasets.load_iris(return_X_y=True, as_frame=True)
#print(iris[1])

The main difference is in the format used to store data: numpy array (version 0.22 and below) or pandas dataframe/series (version 0.23 and above).
To make sure we use the pandas data structure, we can apply the following little conversion (instead of relying on a higher level function):

In [ ]:
import pandas as pd

iris = sklearn.datasets.load_iris()
iris.data = pd.DataFrame(iris.data, columns=iris.feature_names) #converting numpy array -> pandas DataFrame
iris.target = pd.Series(iris.target) #converting numpy array -> pandas Series

The variable `iris` is now a [bunch object](https://scikit-learn.org/stable/modules/generated/sklearn.utils.Bunch.html#sklearn.utils.Bunch) and contains all the required data (via `attributes`: `.data`, `.target`; also with the commented-out syntax above we'd get a bunch object, via indexes `[0]`, `[1]`).

In [ ]:
#uncomment the following instruction to print a detailed description
#of the Iris dataset, here omitted for compactness
print(iris.DESCR)

The dataset describes 150 flower samples, belonging to three species of Iris (Iris setosa, Iris virginica and Iris versicolor). Four features were measured from each sample: the length and the width of the sepals and of the petals, in centimeters. Features are found in the attribute `.data` of the returned bunch object:

In [ ]:
print('Shape of the feature table: ' + str(iris.data.shape))

We can take a look at the actual numbers:

In [ ]:
print(iris.data)

As said above, each of the 150 lines represents a different flower, each belonging to one of the three Iris species. The species will be our target, the class that we are trying to predict. Let's take a look into the dataset:

In [ ]:
print('Shape of the target variable: ' + str(iris.target.shape))
print('Names for each class: ' + str(iris.target_names))

#using Counter object to print a tally of the classes
from collections import Counter
print('Numerosity for each class: ' + str(Counter(iris.target)))

Classes are represented via a numeric index: 0 for *setosa*, 1 for *versicolor*, 2 for *virginica*. The samples are presented in order, with the first 50 samples being *setosa*, then 50 *versicolor* and the last 50 being *virginica*.

Always when working with a new datasets it is importat to plot the data if possible. We are unfortunately talking about a 5-dimensional dataset (the four features + the target class) which is not easily representable. One solution in these cases is to slice a subset of the whole dataset.

In the following code we'll plot two features at a time, plus the class.

In [ ]:
#change these two values to plot different features, remembering the numbering:
# 0 : sepal length (cm)
# 1 : sepal width (cm)
# 2 : petal length (cm)
# 3 : petal width (cm)
feature_x = 0
feature_y = 1

#old reliable pyplot!
import matplotlib.pyplot as plt

#starting a new plot
fig, ax = plt.subplots()

#adding data in three bunches of 50, once per class
ax.scatter(x=iris.data.iloc[0:50,feature_x],    y=iris.data.iloc[0:50,feature_y],    c='red',   label=iris.target_names[0])
ax.scatter(x=iris.data.iloc[50:100,feature_x],  y=iris.data.iloc[50:100,feature_y],  c='green', label=iris.target_names[1])
ax.scatter(x=iris.data.iloc[100:150,feature_x], y=iris.data.iloc[100:150,feature_y], c='blue',  label=iris.target_names[2])

#the axis names are taken from feature names
ax.set_xlabel(iris.feature_names[feature_x])
ax.set_ylabel(iris.feature_names[feature_y])

#adding the legend and printing the plot
ax.legend()
plt.show()

The plot shows clearly that setosa is quite separate from the other two classes. Even chosing other features for the plot the general result is similar.

To be totally frank, this dataset is quite simple. In fact even if it's not possible to easily plot everything, using the four features most classifier can reach very close to 100% accuracy when trying to separate Setosa from the other species.

To make things a little more interesting we decide to **renounce to half of our features**, **using only the first two** columns. Moreover, we **join together Setosa and Versicolor**. In other words, we want a classifier able to discriminate virginica (which becomes the new class "1") from the other irises (which all together become the new class "0"):

In [ ]:
#simplifly the problem: less classes, less features
features = iris.data.iloc[:, 0:2]
target = iris.target

#updating class labels. To makes things difficult we put together old classes 0 and 1
#in a new class (non virginica) and keep old class 2 (virginica) as new class 1.
#For an easier problems put together versicolor and virginica and keep setosa by itself
j = 100 ## split: 50 for setosa vs versicolor+virginica, 100 for setosa+versicolor vs virginica
target[0:j] = 0
target[j:150] = 1

Let's take a look at the new dataset:

In [ ]:
#starting a new plot
fig, ax = plt.subplots()

#adding data in two bunches
ax.scatter(x=features.iloc[0:j,0],   y=features.iloc[0:j,1],   c='red',  label='Not virginica')
ax.scatter(x=features.iloc[j:150,0], y=features.iloc[j:150,1], c='blue', label='virginica')

#the axis names are taken from feature names
ax.set_xlabel(iris.feature_names[feature_x])
ax.set_ylabel(iris.feature_names[feature_y])

#adding the legend and printing the plot
ax.legend()
plt.show()

Things are getting interesting! This is now a difficult problem and there is no clear cut solution. Let's proceed.

# Training and validation sets

Each time there is some kind of "learning" involved we need to split our data. A subset will be used for training, and a subset will be used for validation. (there may be room for another subset, the "test set", but we are not talking about it now).

In our current dataset the samples are sorted by class: the first 100 are from "Not virginica" class, and the remaining 50 are from virginica. We want to keep this 2:1 proportion (roughly) the same in both train and validation set.

In [ ]:
#selecting the first 100 samples for training would be a bad choice...
print(Counter(iris.target[0:100]))

To do so we are going to use what is called a [stratified approach](https://machinelearningmastery.com/cross-validation-for-imbalanced-classification/) using a [StratifiedShuffleSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html) object from Sci-kit learn:

In [ ]:
#we want to have the same proportion of classes in both train and validation sets
from sklearn.model_selection import StratifiedShuffleSplit

#building a StratifiedShuffleSplit object (sss among friends) with 20% data
#assigned to validation set (here called "test")
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

#the .split() method returns (an iterable over) two lists which can be
#used to index the samples that go into train and validation sets
for train_index, val_index in sss.split(features, target):
    features_train = features.iloc[train_index, :]
    features_val   = features.iloc[val_index, :]
    target_train   = target[train_index]
    target_val     = target[val_index]

#let's print some shapes to get an idea of the resulting data structure
print(features_train.shape)
print(features_val.shape)
print(target_train.shape)
print(target_val.shape)

print(type(features_train))
print(type(target_train))

In [ ]:
features_train.head()

It appears that we are using 20% of our data (30 out of 150) for the validation set and the other 80% for the train set.

In [ ]:
print(Counter(target_train))
print(Counter(target_val))

# Logistic regression using Keras

While most statistical packets implement some form of logistic regression in this exercise we are interested in using Keras, which is a library aimed to (Deep) Neural Networks.  
Actually logistic regression plays an important role in neural networks and it's typically used in the last (or second to last) layer of a classifier. For more details on how to use keras a good starting point is the [documentation on training and evaluation](https://www.tensorflow.org/guide/keras/train_and_evaluate).

Our neural network will be very easy and very minimal, and will be comprised of only one node (neuron) implementing both regression (linear combination of weighted input variables + bias term) and sigmoid function.

<img src="https://raw.githubusercontent.com/ne1s0n/coding_excercises/master/data/sigmoid_neuron.png">

We are now ready to build the model!

### Model set-up

In [ ]:
activation_function = 'sigmoid'
optimizing_method = 'rmsprop'
loss_function = 'binary_crossentropy'

In [ ]:
#we are building a "sequential" model, meaning that the data will
#flow like INPUT -> ELABORATION -> OUTPUT. In particular, we will
#not have any loops, i.e. our output will never be recycled as
#input for the first layer
from keras.models import Sequential

#a "dense" layer is a layer were all the data coming in are connected
#to all nodes. In our case there is only one node in the layer, and
#it receives all the features
from keras.layers import Dense

# 2-class logistic regression in Keras
model = Sequential()
model.add(Dense(1, activation='sigmoid', input_dim=features_train.shape[1]))

#the model is declared, but we still need to compile it to actually
#build all the data structures
model.compile(optimizer=optimizing_method, loss=loss_function)

Let's take a look inside the model:

In [ ]:
print(model.summary())

Keras informs us that there are three trainable parameters (W1, W2, B), and a single node. The output is a single number. Excellent.

We have now prepared everything we need and are ready to train the model on our data. It's an iterative process that cycles many times through what are called `epochs`. We'll start with ten:

In [ ]:
history = model.fit(features_train, target_train, epochs=10, validation_data=(features_val, target_val))

We asked for ten epochs and the network did just that. At each iteration the network is trying really hard to minimize a [value called "loss"](https://keras.io/api/losses/). The specifics are defined by our choice of loss function (we selected `binary_crossentropy`). The basic idea is that the smaller the loss the better the fit.

Note that the network minimizes the loss on the training set and does not use the validation set during the learning process. It can however measure the loss on the validation set to give us an idea on how well it can generalize on new data.

It's handy at this point to define a function that takes in the `history` object returned by `.fit()` and plots it:

In [ ]:
#function to take a look at losses evolution
def plot_loss_history(h, title):
    plt.plot(h.history['loss'], label = "Train loss")
    plt.plot(h.history['val_loss'], label = "Validation loss")
    plt.xlabel('Epochs')
    plt.title(title)
    plt.legend()
    plt.show()

In [ ]:
plot_loss_history(history, 'Logistic (10 epochs)')

The good news is that loss just goes down, both in train and validation set. We can keep training - without recompiling, we just add new epochs to our network.

In [ ]:
#putting verbose to 0 to avoid filling the screen
history2 = model.fit(features_train, target_train, epochs=190,
                     validation_data=(features_val, target_val), verbose=0)

Let's see if we improved:

In [ ]:
#putting together the whole history
history.history['loss'] += history2.history['loss']
history.history['val_loss'] += history2.history['val_loss']

#and plotting again
plot_loss_history(history, 'Logistic (500 epochs)')

This is very informative: losses keep shrinking, meaning that the network keeps improving. However after a first phase of steep improvement the gain for each epoch slows down considerably.

Moreover, we now see a clear difference between train and validation set. This means that, while the network keeps improving, its performances on new data are expected to be worse than those on the training data.

Now, we could ask: what happens if we keep training for a long time? We have prepared the code for 10000 epochs, but it takes a long time to run, and it's faster if we simply show the saved results (but you can try to run it putting the following flag to `True`):

In [ ]:
do_10000_epochs = False

In [ ]:
#what happens if we keep going for a (very) long time?
if (do_10000_epochs):
    #train for 10000 epochs, just to show how the model evolves
    history3 = model.fit(features_train, target_train, epochs=9500,
                         validation_data=(features_val, target_val), verbose=0)

    #putting together the whole history
    history.history['loss'] += history3.history['loss']
    history.history['val_loss'] += history3.history['val_loss']

    #and plotting again
    plot_loss_history(history, 'Logistic (10000 epochs)')

Our pre-recorded results look like this:

![regression_10000_epochs](https://raw.githubusercontent.com/ne1s0n/coding_excercises/master/data/regression_loss10000.png)

It appears that there is constant, slow improvement on training set. Improvement on validation set is slower, and if we had the patience to go for a veeeery long time the orange curve would become completely flat.

# Decision Boundary

The assignment asks us to plot the [decision boundary](https://en.wikipedia.org/wiki/Decision_boundary), i.e. a representation in the feature space of the criterions the model is using to classify your data. For this task we'll use the [mlxtend module](http://rasbt.github.io/mlxtend/), which unfortunately does not come with the standard installation. Let's add it!

In [ ]:
!pip install mlxtend ##execute shell command

We can now import `mlxtend` package safely.

In [ ]:
from mlxtend.plotting import plot_decision_regions

#we'll print the training set
plot_decision_regions(X = features_train.to_numpy(), y = target_train.to_numpy(), clf=model)
plt.title('Decision boundary for 0 (non virginica) vs 1 (virginica)')
plt.xlabel(iris.feature_names[feature_x])
plt.ylabel(iris.feature_names[feature_y])
plt.show()

The decision boundary is linear, as expected by logistic regression. This means that all samples in the pink area will be classified as 1 (virginica) and all points in the blue area be considered 0 (non virginica).

Note that relatively many virginica samples are in the blue area compared to the numer of non-virginica present in the pink area. Also note that, roughly speaking, the regressor assigned a wider area to non-virginica. This is a direct consequence of having an unbalanced dataset: two-thirds of the samples are non-virginica (blue squares) and one-third are virginica (red triangles). **The resulting regressor is polarised** toward the more numerous class.

# Actual predictions

Any model is only useful when it's used to predict new, unknown data. In fact the whole validation set was put apart and not really used for training for this specific reason.

Luckily, it's very easy to apply a trained model to new values via the [predict() method](https://keras.io/api/models/model_training_apis/#predict-method).

**Question for you: what do these predictions (below) represent?**

In [ ]:
predictions = model.predict(features_val)
print(predictions)

We can now compare the list of prediction (sometimes called Ŷ) with the true classes from our validation set (sometimes called Y).

In [ ]:
#starting a new plot
fig, ax = plt.subplots()

#adding data in two bunches
ax.scatter(x=range(30), y=predictions, c='red',  label='predictions')
ax.scatter(x=range(30), y=target_val,  c='blue', label='true classes')

#adding a horizontal line at quote 0.5, to represent the decision boundary
ax.hlines(y=0.5, xmin=0, xmax=29, colors='black')

#adding the legend and printing the plot
ax.legend(loc='upper right')
plt.show()

Note that the vast majority of predictions is below the 0.5 line (recall that the closer a prediction is to 0 or to 1 the higher the prediction confidence).

This confirms our suspicions: the regressor prefers to assign new data to the 0 (non virginica) class.

Below, we use the **0.5 threshold** to assign samples to the two classes (0 $\rightarrow$ `non-virginica`; 1 $\rightarrow$ `virginica`.

Then, we compare predicted and true classes in the test set


In [ ]:
predicted_class = np.where(predictions > 0.5, "virginica", "non-virginica")
target_class = np.where(target_val == 1, "virginica", "non-virginica")
target_class = target_class.reshape(len(target_class),1)

results = target_class == predicted_class

unique, counts = np.unique(results, return_counts=True)
print("\nN. of mismatches (0s) and matches (1s): ")
print(np.asarray((unique, counts)).T)

From the number of **matches** (`True` comparison $\rightarrow$ 1s) and **mismatches** (`False` comparison $\rightarrow$ 0s) we can calculate the total **error rate**:

In [ ]:
if len(counts) > 1:
   n_matches = counts[1]
   n_mismatches = counts[0]
else:
  n_matches = counts[0]
  n_mismatches = 0

error_rate = n_mismatches/(n_mismatches+n_matches)
print("Error rate is: {:.2f}".format(round(error_rate,4)))

From the error rate we can derive the **overall accuracy**, by simply taking `1 - error rate`:

In [ ]:
print("Accuracy is: ", 1 - error_rate)

#### Confusion matrix

Besides the overall error rate, we need to look at the accuracy in the subclasses. This is especially important when data are unbalanced. We can have a more comprehensive assessment of the performance of a classification model by looking at the <a href="https://en.wikipedia.org/wiki/Confusion_matrix">confusion matrix</a>.

Let's start by looking closer at the vectors of observed and predicted labels:

In [ ]:
target_class[0:5]

In [ ]:
np.unique(target_class, return_counts=True)

In [ ]:
predicted_class[0:5]

In [ ]:
np.unique(predicted_class, return_counts=True)

Now we are ready to build and examine our **confusion matrix**:

In [ ]:
from sklearn.metrics import confusion_matrix

labels = ['non-virginica','virginica']
con_mat_df = confusion_matrix( y_true = target_class, y_pred = predicted_class, labels=labels) #true are rows, predicted are columns
pd.DataFrame(
    con_mat_df,
    index = ['true:'+x for x in labels],
    columns = ['pred:'+x for x in labels])

**Question for you: from the above results, how do we calculate the TPR?**

# Exercise: do it yourself! (optional)

In the above illustration we used one keras node with sigmoid activation function to perform logistic regression on a binarised version (two classes) of the `Iris` dataset. Only two features were used.

It is now time for you to try write your own code!

There are several things that can be tried as variations on the above code:

* what is the effect of including the other two features, that we excluded above? Does performance improve? What happens to decision boundary?
* when implementing the network we used [rmsprop optimizer](https://keras.io/api/optimizers/rmsprop/) and [binary_crossentropy](https://keras.io/api/losses/probabilistic_losses/#binarycrossentropy-class) loss function, but other [optimizers](https://keras.io/api/optimizers/) and [loss functions](https://keras.io/api/losses/) are available and their effect could be explored


We first set the path (get the data and do a little preprocessing). Then we'll tackle each step separately: first on your own, then together.

In [ ]:
iris = sklearn.datasets.load_iris()
iris.data = pd.DataFrame(iris.data, columns=iris.feature_names) #converting numpy array -> pandas DataFrame
iris.target = pd.Series(iris.target) #converting numpy array -> pandas Series

In [ ]:
features = iris.data
target = iris.target

In [ ]:
features

In [ ]:
j = 100 ## split
target[0:j] = 0  ## non-virginica
target[j:150] = 1 ## virginica

In [ ]:
np.unique(target, return_counts=True)

#### Feature normalization?

How would you do it?

In [ ]:
## your code here

### Training and validation sets

Define your split

In [ ]:
## your code here

### Configuration of (hyper)parameters

Set your parameters

In [ ]:
## your code

### Building and compile the model

Build your neural network logistic regression model, then compile it:

In [ ]:
## your code here

**Question for you**: How many parameters does the model have? Why?

### Fit (train) the model

In [ ]:
## your code here

### Model evaluation

Get predictions and evaluate the performance of the model:

In [ ]:
## your code here

#### On your own (super optional!)

* to improve our understanding of logistic regression we could implement it from scratch, without using keras, following the [steps detailed here](https://towardsdatascience.com/a-logistic-regression-from-scratch-3824468b1f88) (but only if we are very brave)

In [ ]:
## your code here